In [42]:
review_features = {}
review_features["phone"] = ["phone","smartphone","device","frame","quality"]
review_features["seller"] = ["seller","order","store","payment"]
review_features["delivery"]= ["delivery","custom","track","purchase","region","packaging","parcel","box","air","post","package","shipping","condition","damage"]
review_features["service"]=["problem","dispute","complaint"]
review_features["case"]=["film","case","glass","silicone","cover"]
review_features["price"]=["money","price"]
review_features["screen"]=["screen","display","resolution"]
review_features["adapter"]=["adapter"]
review_features["battery"]=["battery","charge","charger"]
review_features["camera"]=["camera","photo","picture"]
review_features["sound"]=["sound","speaker"]
review_features["headphone"]=["headphone"]
review_features["fingerprint"]=["fingerprint"]
review_features["color"]=["color"]
review_features["firmware"]=["firmware"]

In [43]:
import requests
base_url = 'https://m.aliexpress.com/ajaxapi/EvaluationSearchAjax.do?type=all&pageSize=80&country=US'
def importReviews(doc_complete, product_id):
    totalPage = 10000000000000000000
    n = 1
    while (n<=totalPage):
        #print(n)
        url = base_url + "&productId="+str(product_id)+"&index="+str(n)
        #print(url)
        resp = requests.get(url)
        if resp.status_code != 200:
            # This means something went wrong.
            raise ApiError('GET /EvaluationSearchAjax/ {}'.format(resp.status_code))
        resp_json = resp.json()
        doc_complete_n = [review["buyerTranslationFeedback"] 
                          for review in resp_json["evaViewList"] 
                          if "buyerTranslationFeedback" in review.keys() 
                          #and float(review["buyerEval"])<50
                         ]
        #print([float(review["buyerEval"])<50 for review in resp_json["evaViewList"]])
        doc_complete= doc_complete + doc_complete_n
        if (n==1):
            #print(resp_json)
            totalPage = int(resp_json["totalPage"])
            print(str(product_id)+" totalPage "+str(totalPage))
        print(".", end='')    
        n=n+1
    return doc_complete

In [44]:
#32800399551 doogee schießen 2 handy 2 gb - rating 2,7/5
#Xiaomi Redmi 4 Pro Prime
#  32752699189 Doogee x5 max 
# 32757712697 lenovo a5600 
# 32798494733 Lenovo ZUK Z2 
# 32791453530 ONEPLUS 3 T A3003 
# 32671947390  Blackview BV6000 
# 32719969124 Xiaomi Mi Max Mimax Pro 
# 32799364813 Oukitel U20 Plus Android 6.0
# 32611785137 Xiaomi Mi5 M5 Mi 5 Prime
product_ids = [32796300206,32763320059,32792320034,32766086429,32790980431,32721716490,32611785137,32769974242,32662806201]
doc_complete = []
for product_id in product_ids:
    doc_complete=importReviews(doc_complete,product_id)
print("count of reviews: "+ str(len(doc_complete)))

32796300206 totalPage 76
............................................................................count of reviews: 1018


In [45]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    #stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    #punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    tokens = word_tokenize(doc.lower())
    normalized = " ".join(lemma.lemmatize(token) for token in tokens if token not in stop and token not in exclude)
    return normalized

doc_clean = [clean(doc) for doc in doc_complete]   
print(doc_clean)

['phone came quickly within two week use within 4 day positive emotion cause time came third charging phone stopped charging change usb cable charging pc reset factory help dialogue seller lasted 2 week description problem sending video end offered try change usb cable essentially problem similar solution problem new phone offered given diagnosis turned loose contact battery charging board connecting back everything worked phone began charge recharged stop know fix problem real thanks diagnosis especially thank reliable new phone manufacturer `` buying get ready adventure ... ...', 'order dispatch 3 day 17 day moscow good pure gold color without hint pink camera ok read review come marriage play anything anywhere checked office site original phone test chased maybe add review updated air work quickly fingerprint 9/10 even put crooked crumpled bumper phone bigger cutout connector phone nothing simple film use buy another one probably take good film 4 h t. k. apparently glass vendor work

In [46]:
import nltk

full_text = " ".join([review for review in doc_clean])

tokens = nltk.word_tokenize(full_text)

tagged = nltk.pos_tag(tokens)
print(tagged)

[('phone', 'NN'), ('came', 'VBD'), ('quickly', 'RB'), ('within', 'IN'), ('two', 'CD'), ('week', 'NN'), ('use', 'NN'), ('within', 'IN'), ('4', 'CD'), ('day', 'NN'), ('positive', 'JJ'), ('emotion', 'NN'), ('cause', 'NN'), ('time', 'NN'), ('came', 'VBD'), ('third', 'JJ'), ('charging', 'VBG'), ('phone', 'NN'), ('stopped', 'VBD'), ('charging', 'VBG'), ('change', 'NN'), ('usb', 'JJ'), ('cable', 'NN'), ('charging', 'VBG'), ('pc', 'NN'), ('reset', 'NN'), ('factory', 'NN'), ('help', 'NN'), ('dialogue', 'VB'), ('seller', 'NN'), ('lasted', 'VBD'), ('2', 'CD'), ('week', 'NN'), ('description', 'NN'), ('problem', 'NN'), ('sending', 'VBG'), ('video', 'JJ'), ('end', 'NN'), ('offered', 'VBD'), ('try', 'NNS'), ('change', 'VBP'), ('usb', 'JJ'), ('cable', 'NN'), ('essentially', 'RB'), ('problem', 'NN'), ('similar', 'JJ'), ('solution', 'NN'), ('problem', 'NN'), ('new', 'JJ'), ('phone', 'NN'), ('offered', 'VBD'), ('given', 'VBN'), ('diagnosis', 'NN'), ('turned', 'VBD'), ('loose', 'JJ'), ('contact', 'JJ'), (

In [47]:
#nouns = [token for token, pos in tagged if pos.startswith('N')]
#print(nouns)
  	
grammar = r"""
  NOUN_PHRASE:  {<CD|IN|DT|JJ|PP|RB|\$>*<N.*>+<V.*>*<CD|IN|DT|JJ|PP|RB|\$>*<N.*>*}  
"""
cp = nltk.RegexpParser(grammar)
noun_phrase_tagged = cp.parse(tagged)
print(noun_phrase_tagged)
#doc_complete = nouns

(S
  (NOUN_PHRASE
    phone/NN
    came/VBD
    quickly/RB
    within/IN
    two/CD
    week/NN
    use/NN)
  (NOUN_PHRASE
    within/IN
    4/CD
    day/NN
    positive/JJ
    emotion/NN
    cause/NN
    time/NN)
  came/VBD
  third/JJ
  charging/VBG
  (NOUN_PHRASE phone/NN stopped/VBD charging/VBG change/NN)
  (NOUN_PHRASE
    usb/JJ
    cable/NN
    charging/VBG
    pc/NN
    reset/NN
    factory/NN
    help/NN)
  dialogue/VB
  (NOUN_PHRASE
    seller/NN
    lasted/VBD
    2/CD
    week/NN
    description/NN
    problem/NN)
  sending/VBG
  (NOUN_PHRASE video/JJ end/NN offered/VBD try/NNS)
  change/VBP
  (NOUN_PHRASE usb/JJ cable/NN essentially/RB problem/NN)
  (NOUN_PHRASE similar/JJ solution/NN problem/NN new/JJ phone/NN)
  offered/VBD
  given/VBN
  (NOUN_PHRASE
    diagnosis/NN
    turned/VBD
    loose/JJ
    contact/JJ
    battery/NN)
  charging/VBG
  (NOUN_PHRASE board/NN connecting/VBG)
  back/RP
  (NOUN_PHRASE everything/NN worked/VBD phone/NN)
  began/VBD
  (NOUN_PHRASE charge

In [48]:
def traverse(t,noun,phrases):
    try:
        t.label()
    except AttributeError:
        n=0
    else:
        if t.label() == 'NOUN_PHRASE': 
            phrase = " ".join([word for (word,pos) in t.leaves()])
            if noun in phrase.split():
                phrases.append(phrase)
                #print(phrase)
        for child in t:
            traverse(child,noun,phrases)
    return phrases

from nltk.sentiment.vader import SentimentIntensityAnalyzer

dict_noun_abs_sentiment = {}
for noun in review_features.keys():
    phrases=[]
    phrases=traverse(noun_phrase_tagged,noun,phrases)
    print(phrases)
    #doc_complete = nouns

    sid =  SentimentIntensityAnalyzer()
    if len(phrases)!=0:
        print(noun)
        sentiments = sum(sid.polarity_scores(phrase)["compound"] for phrase in phrases)/len(phrases)
        print(noun + " has avg sentiment " + '{:01.2f}'.format(sentiments))
        dict_noun_abs_sentiment[noun]=sentiments
        print()

['phone came quickly within two week use', 'phone stopped charging change', 'similar solution problem new phone', 'everything worked phone', 'especially thank reliable new phone manufacturer', 'office site original phone test', 'bumper phone', 'cutout connector phone nothing simple film use buy another one probably', 'general phone happy recommend seller cool phone money', 'except protective glass complete fit edge phone', 'little bent edge glass stick great phone', 'phone charge hold well camera bad satisfied purchase', 'value 18 description android phone film adapter socket gift thank box', 'phone film sealed apparently', 'checked flash device smart phone camera', 'мибэндом phone turned apparently work', 'touch подглючал phone probably norm', 'already news strain мифорумов forum wish неуемное write phone number', 'update ordering brilliant phone money case gift delivery 18 day', 'quickly 11-13 day complaint phone', 'stitching-глобалка сбер stood without problem recommend phone', 'key

phone has avg sentiment 0.33

['seller lasted 2 week description problem', 'general phone happy recommend seller cool phone money', 'good device thank seller fast shipment track отсокживался', 'qualitatively squeak backlash recommend seller product', 'thank excellent seller photo well', 'kant stick recommend product seller everything fine', 'edge display bulletproof kant stick recommend product seller everything', 'seller good smartphone', 'statement description recommend seller model seller', '10-five reach seller checked first tube store everything', 'gift pleasant residue seller', 'seller let little expensive quality speed', 'alibonus воспользуетесь reference com/u 677012 seller', 'everything work thank seller sent next day order otslezhtvalas', 'excellent smartphone price category ordered seller', 'seller sends always quickly problem battery', 'lain week local post office seller communicate track track problem custom everything fine', 'creak spasibo seller happy purchase order', 't

seller has avg sentiment 0.38

['fine excellent device money satisfied delivery', 'update ordering brilliant phone money case gift delivery 18 day', 'delivery fast phone', 'delivery 11 day phone super', 'thank seller store goldway fast reliable delivery', 'unit fast delivery', 'megabystro 11 day quick delivery', 'good phone fast delivery', 'protective film delivery time good luck', 'place delivery june 13 2017 17:21 636017 северск', 'center june 11 2017 00:30 102320 domodedovo delivery russia june', 'touchscreen delivery fast recommend order', 'delivery mail singapore 3 week khabarovsk', '5 star delivery excellent 2 week yamalo-nenets', 'lg functionality program playmarket work unsatisfactory delivery belgorod', 'silicone case plyonochku screen delivery yaroslavl little 2 week parcel', 'fast delivery phone fire daughter loved fast shipping top', '6500 6000 suitable fast delivery tyumen 12 day product', 'excellent quality found shortcoming fast delivery tyumen', 'upakovka capital came w

['gift silicone case film adapter charging came', 'update ordering brilliant phone money case gift delivery 18 day', 'phone headphone prodavets put gift silicone case film screen', 'silicone case course crumpled rough scratchy', 'budget smartphone metal case came quickly gift film adapter plug', '10.07 ярославскию area put adapter silicone case film', '43 parrot ordered cover glass case', 'repair helped another case', 'excellent choice included silicone case glass', 'case', 'silicone case seller gave soviet outlet adapter charger', 'silicone case plyonochku screen delivery yaroslavl little 2 week parcel', 'glass silicone case case tolerable first time', 'budget smartphone metal case capacious battery gift adapter plug', 'good catch wi fi mt 4g v gift case film', 'film screen adapter socket euro silicone case', 'tyrnete full information case film included simple first time', 'choose language country seller put gift plug silicone case', 'first time history shopping ali came marriage case

['value 18 description android phone film adapter socket gift thank box', 'adapter russian curved edge display bulletproof', 'package air included adapter russian', 'gift silicone case film adapter charging came', 'put adapter outlet ordinary film screen box', 'high quality wrinkle gift bag adapter outlet film screen color', 'adapter outlet gift put film phone', 'manually tool phone seller sent adapter socket', 'budget smartphone metal case came quickly gift film adapter plug', 'fine gift wrap put adapter seller communicate recommend', 'complete euro adapter russian outlet film screen headphone', '10.07 ярославскию area put adapter silicone case film', 'separately pleased adapter razetku sturdy hold general satisfied excellent seller', 'seller put gift plenochku adapter', 'first time useful adapter', 'gift spare protective film adapter plug', 'adapter evrorazetku took care phone', 'gift seller put adapter euro protective film', 'standard phone charger adapter film screen gift order sen

['dynamic sound good clean thing camera', 'height team phone simply gorgeous sound', 'flawlessly afraid speaker conversation quiet clear sound', 'really comfortable speaker give clear high-quality sound call interlocutor heard', 'small pleasant bass headphone also good sound', 'different pleased sound phone easy nice cold hand', 'firmware updated surprised sound', 'safe sound bad protective film', 'happy phone sound clear picture', 'also high level model powerful battery sound', 'great sound music hearing dynamic fingerprint scanner work', 'bad sound certainly loud prodavets', 'sound headphone rule big issue', 'excellent sound video speed thanks seller excellent service road', 'safe sound custom problem 5 star advise everyone book', 'safe sound smart telefonchik loud screen bright first press', 'great phone look hand convenient fotki crisp sound', 'safe sound work fine within 2 week thank seller recommend thank', 'decent design phone screen good decent sound', '20 day everything fine p

In [49]:
print( [ (key,'{:01.2f}'.format(val)) for (key,val) in dict_noun_abs_sentiment.items() ])

[('phone', '0.33'), ('seller', '0.38'), ('delivery', '0.32'), ('service', '0.30'), ('case', '0.24'), ('price', '0.33'), ('screen', '0.34'), ('adapter', '0.29'), ('battery', '0.24'), ('camera', '0.34'), ('sound', '0.51'), ('headphone', '0.16'), ('fingerprint', '0.48'), ('color', '0.34'), ('firmware', '0.21')]
